In [1]:
from IPython.display import FileLink

In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(DESeq2)
library(phyloseq)
library(plyr); library(dplyr)
library(ggplot2)
library(doParallel)
library(foreach)
library(tidyr)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:stats’:

    xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, as.vector, cbind, colnames,
    do.call, duplicated, eval, evalq, Filter, Find, get, intersect,
    is.unsorted, lapply, Map, mapply, match, mget, order, paste, pmax,
    pmax.int, pmin, pmin.int, Position, rank, rbind, Reduce, rep.int,
    rownames, sapply, setdiff, sort, table, tapply, union, unique,
    unlist, unsplit

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package:

###These files are from the [assign taxonomy notebook](./Assign_taxonomy.ipynb), [tree notebook](./Tree.ipynb), [merge mappers notebook](./Merge_mappers.ipynb) 

In [4]:
%%R
physeq = import_biom("/var/seq_data/priming_exp/data/otu_table_wtax.biom", "/var/seq_data/priming_exp/data/otusn.tree")

sample.data = read.table("/var/seq_data/priming_exp/data/sample_data_all.csv",
                         sep = ",", header = TRUE, colClasses = c("Density" = "numeric"),
                         na.strings = c("nan", ""))
rownames(sample.data) = sample.data$Sample
sample.data = sample_data(sample.data)

sample_data(physeq) = sample.data

physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 10361 taxa and 467 samples ]
sample_data() Sample Data:       [ 467 samples by 12 sample variables ]
tax_table()   Taxonomy Table:    [ 10361 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 10361 tips and 10360 internal nodes ]


In [5]:
%%R
md = sample_data(physeq)
physeq.rep = prune_samples(!is.na(md$rep), physeq)
physeq.rep

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 10361 taxa and 60 samples ]
sample_data() Sample Data:       [ 60 samples by 12 sample variables ]
tax_table()   Taxonomy Table:    [ 10361 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 10361 tips and 10360 internal nodes ]


### Get log2foldchange for CC

In [6]:
%%R
l2fc.threshold = 0.75
density.min = 1.7125
density.max = 1.755
sparsity.threshold = 0.45
                           
get_Ps = function(physeq.obj, Rep) {
    
    physeq.md = sample_data(physeq.obj)
    p = prune_samples((physeq.md$rep %in% c(Rep, "control_rep"))&
                      (physeq.md$Density >= density.min)&
                      (physeq.md$Density <= density.max), physeq.obj)
    p.thresh = filter_taxa(p, function(x) sum(x > 0) > sparsity.threshold * length(x), TRUE)
    dds = phyloseq_to_deseq2(p.thresh, ~Treatment)
    dds = DESeq(dds, quiet = TRUE, fitType = "local")
    theta = l2fc.threshold
    r = results(dds, independentFiltering = FALSE)
    r$OTU = rownames(r)
    beta = r$log2FoldChange
    betaSE = r$lfcSE
    p = pnorm(beta, theta, betaSE, lower.tail = FALSE)
    r$p = p
    r$rep = Rep
    d = data.frame(r[, c("OTU","log2FoldChange", "p", "rep")])
    TT = data.frame(tax_table(p.thresh))
    TT$OTU = rownames(TT)
    d = left_join(d, TT)
    d
}

In [7]:
%%R
l2fc.df.rep = ldply(c("label_rep_06","label_rep_07"), get_Ps, physeq.obj = physeq.rep)

converting counts to integer mode
Joining by: "OTU"
converting counts to integer mode
Joining by: "OTU"


In [8]:
%%R
head(l2fc.df.rep)

       OTU log2FoldChange         p          rep    Rank1         Rank2 Rank3
1 OTU.4204    -0.50050740 0.9999949 label_rep_06 Bacteria Acidobacteria DA023
2  OTU.467    -0.01260815 0.9757167 label_rep_06 Bacteria Acidobacteria DA023
3  OTU.905    -0.23943188 0.9236757 label_rep_06 Bacteria Acidobacteria DA023
4 OTU.7253     0.53509768 0.6254603 label_rep_06 Bacteria Acidobacteria DA023
5   OTU.67    -0.25478344 0.9998535 label_rep_06 Bacteria Acidobacteria DA023
6 OTU.4907    -0.18195344 1.0000000 label_rep_06 Bacteria Acidobacteria DA023
                 Rank4 Rank5 Rank6 Rank7 Rank8
1                 <NA>  <NA>  <NA>  <NA>  <NA>
2 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>
3 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>
4 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>
5 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>
6 uncultured_bacterium  <NA>  <NA>  <NA>  <NA>


In [9]:
%%R
l2fc.df.rep.padj = l2fc.df.rep %>%
    group_by(rep) %>%
    mutate(padj = p.adjust(p, method = "BH"))

In [10]:
%%R
write.table(l2fc.df.rep.padj, 
            file = "/var/seq_data/priming_exp/data/l2fc_table_13C100rep.csv",
            row.names = FALSE,
            sep = ",")

In [11]:
%%R -w 400 -h 375
d = l2fc.df.rep.padj %>% select(OTU, rep, 2^log2FoldChange) %>% spread(rep, log2FoldChange)

p = ggplot(d, aes(x = label_rep_06, y = label_rep_07))

p = p + geom_point(shape = 21)

p = p + stat_smooth(method = "lm")

p = p + labs(x = "Replicate 1, l2fc",
             y = "Replicate 2, l2fc")

p = p + theme_bw()

p

Error in dim(ordered) <- c(attr(row_id, "n"), attr(col_id, "n")) : 
  attempt to set an attribute on NULL


In [12]:
%%R 
ggsave(plot = p, filename = "figs/Replicate_PE_scatter.pdf", width = 12, height = 10)

Error in inherits(plot, "ggplot") : object 'p' not found


In [13]:
FileLink("figs/Replicate_PE_scatter.pdf")

/home/ashley/priming_exp/figs/Replicate_PE_scatter.pdf

In [14]:
%%R
fit = lm(label_rep_06 ~ label_rep_07, d)
summary(fit)

Error in is.data.frame(data) : object 'd' not found


In [15]:
%%R
FDR = 0.10

df.sum.sig = l2fc.df.rep.padj %>%
    group_by(OTU) %>%
    summarize(sum_sig = sum(padj < FDR, na.rm = TRUE))

In [16]:
%%R -w 500 -h 375
p = ggplot(left_join(d, df.sum.sig), aes(x = label_rep_06, y = label_rep_07, color = factor(sum_sig)))

p = p + geom_point(shape = 21)

p = p + labs(x = "Replicate 1, l2fc",
             y = "Replicate 2, l2fc", 
             color = "# of\np-values < FDR")

p = p + theme_bw()

p

Error in left_join(d, df.sum.sig) : object 'd' not found


In [17]:
%%R 
ggsave(plot = p, filename = "figs/Replicate_PE_scatter_color.pdf", width = 12, height = 10)

Error in inherits(plot, "ggplot") : object 'p' not found


In [18]:
FileLink("figs/Replicate_PE_scatter_color.pdf")

/home/ashley/priming_exp/figs/Replicate_PE_scatter_color.pdf